# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f89699722b0>
├── 'a' --> tensor([[-1.3977,  0.9155, -0.3972],
│                   [ 1.0971,  1.0379, -0.7297]])
└── 'x' --> <FastTreeValue 0x7f8969972fa0>
    └── 'c' --> tensor([[ 0.7228,  0.1475, -0.5103, -1.5172],
                        [-1.1956, -0.6741,  0.7267, -0.9657],
                        [ 0.3556,  0.1948, -0.1893,  0.2583]])

In [4]:
t.a

tensor([[-1.3977,  0.9155, -0.3972],
        [ 1.0971,  1.0379, -0.7297]])

In [5]:
%timeit t.a

63.8 ns ± 0.135 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f89699722b0>
├── 'a' --> tensor([[-1.7069,  0.5678, -0.5544],
│                   [-0.0411,  0.4777, -0.2856]])
└── 'x' --> <FastTreeValue 0x7f8969972fa0>
    └── 'c' --> tensor([[ 0.7228,  0.1475, -0.5103, -1.5172],
                        [-1.1956, -0.6741,  0.7267, -0.9657],
                        [ 0.3556,  0.1948, -0.1893,  0.2583]])

In [7]:
%timeit t.a = new_value

69.7 ns ± 0.0988 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.3977,  0.9155, -0.3972],
               [ 1.0971,  1.0379, -0.7297]]),
    x: Batch(
           c: tensor([[ 0.7228,  0.1475, -0.5103, -1.5172],
                      [-1.1956, -0.6741,  0.7267, -0.9657],
                      [ 0.3556,  0.1948, -0.1893,  0.2583]]),
       ),
)

In [10]:
b.a

tensor([[-1.3977,  0.9155, -0.3972],
        [ 1.0971,  1.0379, -0.7297]])

In [11]:
%timeit b.a

59.2 ns ± 0.0667 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.0654, -0.4045,  0.8025],
               [ 0.5815,  0.3411,  2.7741]]),
    x: Batch(
           c: tensor([[ 0.7228,  0.1475, -0.5103, -1.5172],
                      [-1.1956, -0.6741,  0.7267, -0.9657],
                      [ 0.3556,  0.1948, -0.1893,  0.2583]]),
       ),
)

In [13]:
%timeit b.a = new_value

496 ns ± 0.0928 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

841 ns ± 1.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.9 µs ± 32.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 1.61 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 1.28 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f88b01dac10>
├── 'a' --> tensor([[[-1.3977,  0.9155, -0.3972],
│                    [ 1.0971,  1.0379, -0.7297]],
│           
│                   [[-1.3977,  0.9155, -0.3972],
│                    [ 1.0971,  1.0379, -0.7297]],
│           
│                   [[-1.3977,  0.9155, -0.3972],
│                    [ 1.0971,  1.0379, -0.7297]],
│           
│                   [[-1.3977,  0.9155, -0.3972],
│                    [ 1.0971,  1.0379, -0.7297]],
│           
│                   [[-1.3977,  0.9155, -0.3972],
│                    [ 1.0971,  1.0379, -0.7297]],
│           
│                   [[-1.3977,  0.9155, -0.3972],
│                    [ 1.0971,  1.0379, -0.7297]],
│           
│                   [[-1.3977,  0.9155, -0.3972],
│                    [ 1.0971,  1.0379, -0.7297]],
│           
│                   [[-1.3977,  0.9155, -0.3972],
│                    [ 1.0971,  1.0379, -0.7297]]])
└── 'x' --> <FastTreeValue 0x7f88b680cdc0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 66.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f88afbabd90>
├── 'a' --> tensor([[-1.3977,  0.9155, -0.3972],
│                   [ 1.0971,  1.0379, -0.7297],
│                   [-1.3977,  0.9155, -0.3972],
│                   [ 1.0971,  1.0379, -0.7297],
│                   [-1.3977,  0.9155, -0.3972],
│                   [ 1.0971,  1.0379, -0.7297],
│                   [-1.3977,  0.9155, -0.3972],
│                   [ 1.0971,  1.0379, -0.7297],
│                   [-1.3977,  0.9155, -0.3972],
│                   [ 1.0971,  1.0379, -0.7297],
│                   [-1.3977,  0.9155, -0.3972],
│                   [ 1.0971,  1.0379, -0.7297],
│                   [-1.3977,  0.9155, -0.3972],
│                   [ 1.0971,  1.0379, -0.7297],
│                   [-1.3977,  0.9155, -0.3972],
│                   [ 1.0971,  1.0379, -0.7297]])
└── 'x' --> <FastTreeValue 0x7f8937fcc0a0>
    └── 'c' --> tensor([[ 0.7228,  0.1475, -0.5103, -1.5172],
                        [-1.1956, -0.6741,  0.7267, -0.9657],
                 

In [23]:
%timeit t_cat(trees)

30.2 µs ± 61.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.7 µs ± 197 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.7228,  0.1475, -0.5103, -1.5172],
                       [-1.1956, -0.6741,  0.7267, -0.9657],
                       [ 0.3556,  0.1948, -0.1893,  0.2583]],
              
                      [[ 0.7228,  0.1475, -0.5103, -1.5172],
                       [-1.1956, -0.6741,  0.7267, -0.9657],
                       [ 0.3556,  0.1948, -0.1893,  0.2583]],
              
                      [[ 0.7228,  0.1475, -0.5103, -1.5172],
                       [-1.1956, -0.6741,  0.7267, -0.9657],
                       [ 0.3556,  0.1948, -0.1893,  0.2583]],
              
                      [[ 0.7228,  0.1475, -0.5103, -1.5172],
                       [-1.1956, -0.6741,  0.7267, -0.9657],
                       [ 0.3556,  0.1948, -0.1893,  0.2583]],
              
                      [[ 0.7228,  0.1475, -0.5103, -1.5172],
                       [-1.1956, -0.6741,  0.7267, -0.9657],
                       [ 0.3556,  0.1948, -0.1893,  0.2583]],

In [26]:
%timeit Batch.stack(batches)

79.5 µs ± 177 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.7228,  0.1475, -0.5103, -1.5172],
                      [-1.1956, -0.6741,  0.7267, -0.9657],
                      [ 0.3556,  0.1948, -0.1893,  0.2583],
                      [ 0.7228,  0.1475, -0.5103, -1.5172],
                      [-1.1956, -0.6741,  0.7267, -0.9657],
                      [ 0.3556,  0.1948, -0.1893,  0.2583],
                      [ 0.7228,  0.1475, -0.5103, -1.5172],
                      [-1.1956, -0.6741,  0.7267, -0.9657],
                      [ 0.3556,  0.1948, -0.1893,  0.2583],
                      [ 0.7228,  0.1475, -0.5103, -1.5172],
                      [-1.1956, -0.6741,  0.7267, -0.9657],
                      [ 0.3556,  0.1948, -0.1893,  0.2583],
                      [ 0.7228,  0.1475, -0.5103, -1.5172],
                      [-1.1956, -0.6741,  0.7267, -0.9657],
                      [ 0.3556,  0.1948, -0.1893,  0.2583],
                      [ 0.7228,  0.1475, -0.5103, -1.5172],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 384 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

336 µs ± 2.44 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
